In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

d:\anaconda\envs\OPENAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
words = open("names.txt" , 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i , s in enumerate(chars)}
stoi['.'] = 0
itos = {i : s for s , i in stoi.items()}
vocab_size = len(itos)
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
import random

block_size = 3
def build_dataset(words):
    X , Y = [] , []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context) , '--->' , itos[ix])
            context = context[1 :] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape , Y.shape) 
    return X , Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr , Ytr = build_dataset(words[:n1])     #train data
Xdev , Ydev = build_dataset(words[n1:n2]) #dev data
Xte , Yte = build_dataset(words[n2:])     #test data

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [37]:
n_emb = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
#当embcat @ W1 + b1的值较大时，tanh趋近于平坦，梯度很小，故影响损失的传递（压缩了梯度）
#神经元可能无法被激活，可能因为初始化或者优化过程使神经元死亡从而无法被激活
#所以希望初始化的预激活值更加趋近于0
C = torch.randn((vocab_size , n_emb) , generator = g)
#此处初始化方法为kaiming_noraml
W1 = torch.randn((n_emb * block_size , n_hidden) , generator = g) * (5/3) * ((n_emb * block_size) ** 0.5)
b1 = torch.randn(n_hidden , generator = g) * 0.01

#希望刚开始的输出趋近于平均分布
W2 = torch.randn((n_hidden , vocab_size) , generator = g) * 0.01
b2 = torch.randn(vocab_size , generator=g) * 0

bngain = torch.ones((1 , n_hidden))
bnbias = torch.ones((1 , n_hidden))
bnmean_running = torch.ones((1 , n_hidden))
bnstd_running = torch.ones((1 , n_hidden))

parameters = [C , W1 , b1 , W2 , b2 , bngain , bnbias]
total_parameters = sum(p.nelement() for p in parameters)
print(total_parameters)

for p in parameters:
    p.requires_grad = True

12297


In [36]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    #mini batch
    ix = torch.randint(0 , Xtr.shape[0] , (batch_size , ) , generator=g)
    Xb , Yb = Xtr[ix] , Ytr[ix]

    emb = C[Xb]
    embcat = emb.view(emb.shape[0] , -1)
    hpreact = embcat @ W1 # + b1 因为有Norm层所以不需要bias
    
    #BatchNorm Layer
    bnmean = hpreact.mean(0 , keepdim=True)
    bnstd = hpreact.std(0 , keepdim=True)
    hpreact = bngain  * (hpreact - bnmean) / bnstd + bnbias
    with  torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmean
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstd
        
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits , Yb)

    for p in parameters:
        p.grad = None
    loss.backward()

    # lr = lrs[i]
    #可以随着训练轮数的增加进行学习率衰减
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    #track stats
    if (i + 1) % 10000 == 0:
        print(f'{i + 1:7d}/{max_steps:7d}: {loss.item():.4f}')
    elif i == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    break

      0/ 200000: 3.3091


In [29]:
@torch.no_grad()
def split_loss(split):
    x , y = {
        'train' : (Xtr , Ytr) , 
        'val' : (Xdev , Ydev) ,
        'test' : (Xte , Yte),
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0] , -1)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits , y)
    print(split , loss.item())

split_loss('train')
split_loss('val')

train 2.1189446449279785
val 2.1612794399261475


In [30]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor(context)]
        h = torch.tanh(emb.view(1 , -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits , dim=1)
        ix = torch.multinomial(probs , num_samples=1 , generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

carmah.
ambrilli.
kimri.
rehtleenan.
saeja.
hubb.
den.
arti.
geni.
ner.
kiah.
maiir.
kaleigh.
ham.
jore.
quint.
sulin.
alianni.
waterry.
jarynix.
